In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns 
from tqdm import tqdm
import random
import os
import keras 

%matplotlib inline

from keras_radam import RAdam
from keras.optimizers import Adam

In [2]:
np.random.seed(7)
random.seed(7)
#tf.random.set_seed(7)

In [3]:
train_ = pd.read_csv('./data/train.csv',index_col='id')
test_ = pd. read_csv('./data/test.csv',index_col='id')

In [4]:
train=train_.copy()
test=test_.copy()

In [5]:
# 기상청 데이터만 추출
x_train = train.loc[:,'X00':'X39']

# standardization을 위해 평균과 표준편차 구하기
MEAN = x_train.mean()
STD = x_train.std()

# 표준편차가 0일 경우 대비하여 1e-07 추가 
x_train = (x_train - MEAN) / (STD + 1e-07)

In [6]:
# RNN 모델에 입력 할 수 있는 시계열 형태로 데이터 변환 
def convert_to_timeseries(df, interval):
    sequence_list = []
    target_list = []
    
    for i in tqdm(range(df.shape[0] - interval)):
        sequence_list.append(np.array(df.iloc[i:i+interval,:-1]))
        target_list.append(df.iloc[i+interval,-1])
    
    sequence = np.array(sequence_list)
    target = np.array(target_list)
    
    return sequence, target

In [7]:
y_columns = ['Y15','Y16']
#y_columns= train.iloc[:,40:-1].columns

In [8]:
# t시점 이전 120분의 데이터로 t시점의 온도를 추정할 수 있는 학습데이터 형성
sequence = np.empty((0, 12, 40))
target = np.empty((0,))
for column in y_columns :
    
    concat = pd.concat([x_train, train[column]], axis = 1)

    _sequence, _target = convert_to_timeseries(concat.head(144*30), interval = 12)

    sequence = np.vstack((sequence, _sequence))
    target = np.hstack((target, _target))

100%|████████████████████████████████████████████████████████████████████████████| 4308/4308 [00:01<00:00, 2505.52it/s]


In [9]:
# convert_to_timeseries 함수를 쓰기 위한 dummy feature 생성
x_train['dummy'] = 0

In [10]:
# train set에서 도출된 평균과 표준편차로 standardization 실시 
test = (test - MEAN) / (STD + 1e-07)

In [11]:
# convert_to_timeseries 함수를 쓰기 위한 dummy feature 생성
test['dummy'] = 0

In [12]:
# train과 test 기간을 합쳐서 120분 간격으로 학습데이터 재구축
x_test, _ = convert_to_timeseries(pd.concat([x_train, test], axis = 0), interval=12)

100%|██████████████████████████████████████████████████████████████████████████| 16260/16260 [00:06<00:00, 2337.76it/s]


In [13]:
# test set 기간인 후반부 80일에 맞게 자르기 
x_test = x_test[-11520:, :, :]

In [14]:
# 만들어 두었던 dummy feature 제거
x_train.drop('dummy', axis = 1, inplace = True)
test.drop('dummy', axis = 1, inplace = True)

In [ ]:
# 간단한 lstm 모델 구축하기 
simple_lstm_model = keras.models.Sequential([
    keras.layers.LSTM(32, input_shape=sequence.shape[-2:]),
    keras.layers.Dense(64, activation='elu'),
    keras.layers.Dense(32, activation='elu'),    
    keras.layers.Dense(1)
])

simple_lstm_model.compile(optimizer='adam', loss='mse')

In [16]:
# loss가 4미만으로 떨어지면 학습 종료 시키는 기능
class myCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs = None):
        if(logs.get('loss') < 0.0001):
            print('\n Loss is under 0.0001, cancelling training')
            self.model.stop_training = True

In [17]:
callbacks = myCallback()

In [ ]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(64, input_shape=sequence.shape[-2:]),
#     keras.layers.Dense(64, activation='elu'),
#     keras.layers.Dense(32, activation='elu'),    
#     keras.layers.Dense(1)
# ])

# simple_lstm_model.compile(optimizer='adam', loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=60, batch_size=128, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

In [ ]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(32, input_shape=sequence.shape[-2:]),
#     keras.layers.Dense(64, activation='elu'),
#     keras.layers.Dense(32, activation='elu'),    
#     keras.layers.Dense(1)
# ])
# simple_lstm_model.compile(optimizer='adam', loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=80, batch_size=128, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

# Epoch 80/80
#  - 1s - loss: 2.5424 - val_loss: 2.6561

In [ ]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(64, input_shape=sequence.shape[-2:], activation='tanh'),
#     keras.layers.Dense(64, activation='elu'),
#     keras.layers.Dense(64, activation='elu'),
#     keras.layers.Dense(64, activation='elu'),
#     keras.layers.Dense(64, activation='elu'),    
#     keras.layers.Dropout(0.05),
#     keras.layers.Dense(1)
# ])
# simple_lstm_model.compile(optimizer='adam', loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=80, batch_size=128, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

# Epoch 80/80
# - 1s - loss: 3.1128 - val_loss: 3.5588

In [ ]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(64, input_shape=sequence.shape[-2:], activation='tanh', return_sequences=True),
#     keras.layers.LSTM(64),
#     keras.layers.Dense(64, activation='elu'),    
#     keras.layers.Dropout(0.05),
#     keras.layers.Dense(1)
# ])
# simple_lstm_model.compile(optimizer='adam', loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=80, batch_size=128, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

# Epoch 79/80
#  - 2s - loss: 2.8924 - val_loss: 3.3730
# Epoch 80/80
#  - 2s - loss: 2.7272 - val_loss: 3.1412


In [ ]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(64, input_shape=sequence.shape[-2:], activation='tanh', return_sequences=True),
#     keras.layers.LSTM(64),
#     keras.layers.Dense(64, activation='elu'),    
#     keras.layers.Dropout(0.05),
#     keras.layers.Dense(1)
# ])
# simple_lstm_model.compile(optimizer='adam', loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=80, batch_size=32, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

# Epoch 78/80
#  - 4s - loss: 2.0523 - val_loss: 2.3489
# Epoch 79/80
#  - 4s - loss: 2.2302 - val_loss: 2.2888
# Epoch 80/80
#  - 4s - loss: 2.0973 - val_loss: 3.1370

In [30]:
# # 간단한 lstm 모델 구축하기 
# simple_lstm_model = keras.models.Sequential([
#     keras.layers.LSTM(256, input_shape=sequence.shape[-2:], activation='tanh'),
# #    keras.layers.LSTM(64, activation='tanh'),
#     keras.layers.Dense(256, activation='elu'),
#     keras.layers.Dropout(0.2),
#     keras.layers.Dense(1)
# ])
# simple_lstm_model.compile(optimizer=Adam(lr=1.e-4), loss='mse')
# # 모델 학습
# simple_lstm_model.fit(sequence, target, epochs=80, batch_size=256, verbose=2,
#     shuffle=False, callbacks = [callbacks], validation_split=0.1)

Train on 7754 samples, validate on 862 samples
Epoch 1/80
 - 5s - loss: 523.9605 - val_loss: 579.0571
Epoch 2/80
 - 4s - loss: 448.9724 - val_loss: 494.2359
Epoch 3/80
 - 4s - loss: 281.8172 - val_loss: 305.9006
Epoch 4/80
 - 5s - loss: 115.0034 - val_loss: 138.7109
Epoch 5/80
 - 4s - loss: 51.7986 - val_loss: 51.8027
Epoch 6/80
 - 4s - loss: 27.6234 - val_loss: 25.0665
Epoch 7/80
 - 4s - loss: 21.2916 - val_loss: 18.2466
Epoch 8/80
 - 4s - loss: 17.6657 - val_loss: 14.9382
Epoch 9/80
 - 4s - loss: 15.1468 - val_loss: 12.5885
Epoch 10/80
 - 4s - loss: 12.8736 - val_loss: 10.5851
Epoch 11/80
 - 5s - loss: 11.4787 - val_loss: 9.6024
Epoch 12/80
 - 5s - loss: 11.0108 - val_loss: 9.0160
Epoch 13/80
 - 5s - loss: 10.3651 - val_loss: 8.6302
Epoch 14/80
 - 4s - loss: 9.9155 - val_loss: 8.3662
Epoch 15/80
 - 5s - loss: 9.5926 - val_loss: 8.1205
Epoch 16/80
 - 4s - loss: 9.3343 - val_loss: 7.9701
Epoch 17/80
 - 4s - loss: 9.0352 - val_loss: 7.9211
Epoch 18/80
 - 4s - loss: 8.9633 - val_loss: 7.

KeyboardInterrupt: 

In [ ]:
# LSTM 레이어는 고정
simple_lstm_model.layers[0].trainable = False

In [ ]:
# fine tuning 할 때 사용할 학습데이터 생성 (Y18)
finetune_X, finetune_y = convert_to_timeseries(pd.concat([x_train.tail(432), train['Y18'].tail(432)], axis = 1), interval=12)

In [ ]:
# # LSTM 레이어는 고정 시켜두고, DNN 레이어에 대해서 fine tuning 진행 (Transfer Learning)
# finetune_history = simple_lstm_model.fit( finetune_X, finetune_y, epochs=20,
#             batch_size=64, shuffle=False, verbose = 2)

In [ ]:
# 예측하기 
finetune_pred = simple_lstm_model.predict(x_test)

In [ ]:
# 제출 파일 만들기
submit = pd.DataFrame({'id':range(144*33, 144*113),
              'Y18':finetune_pred.reshape(1,-1)[0]})

In [ ]:
submit.to_csv('baseline_result.csv', index = False)